In [ ]:
import pandas as pd
from rouge import Rouge 
import numpy as np
import re
import sys
sys.executable

In [ ]:
# read excel file
df = pd.read_excel('../results/falcon-7b-NOT-FOUND-REMOVED-firstpart-4epoch-lr4e-4.xlsx')


In [ ]:
rouge = Rouge()

In [ ]:
f_scores=[]
p_scores=[]
r_scores=[]
structures=[]
cosine_scores=[]
is_includeds = []
report_types = []
for  row in df.itertuples():
   
    rouge_L_f=[]
    rouge_1_p=[]
    rouge_1_r=[]
    is_included = []
    cosine_score=[]
    possible_asnwers = row.answer.split('|')
    possible_asnwers = [answer.strip() for answer in possible_asnwers]
    
    generated_predictions = row.prediction.split('|')
    generated_predictions = [prediction.strip() for prediction in generated_predictions]
    
    p=generated_predictions[0]
    a=possible_asnwers[0]

    if p=="" or a=="" :
        continue
    scores = rouge.get_scores(p, a)
    rouge_1_p.append(scores[0]['rouge-l']['p'])
    rouge_1_r.append(scores[0]['rouge-l']['r'])
    rouge_L_f.append(scores[0]['rouge-l']['f'])
    is_included.append(a in (p) or p in (a))
    
    
    
    
    p_scores.append(np.max(rouge_1_p))
    r_scores.append(np.max(rouge_1_r))
    f_scores.append(np.max(rouge_L_f))
    is_includeds.append(np.max(is_included))
    structures.append(row.structure)
    report_types.append(row.report_type)
    
    

   
result_scores= pd.DataFrame({'f_score':f_scores,'p_score':p_scores,'r_score':r_scores,'structure':structures ,'r_type':report_types,'is_included':is_includeds})

In [ ]:
for report_type in result_scores.r_type.unique():
    print("-----",report_type,'-------')
    for structures in result_scores.structure.unique():
        print("-----",structures,'-------')
        sub_data = result_scores[(result_scores.structure==structures) & (result_scores.r_type==report_type)]
        print("f_score_mean",sub_data.f_score.mean().round(4))
        # print("f_score_std",sub_data.f_score.std().round(4))
        print("p_score_mean",sub_data.p_score.mean().round(4))
        # print("p_score_std",sub_data.p_score.std().round(4))
        print("r_score_mean",sub_data.r_score.mean().round(4))
        # print("r_score_std",sub_data.r_score.std().round(4))
        print("is_included_mean",sub_data.is_included.sum() ,"out of",sub_data.shape[0])


In [ ]:
df = pd.read_excel('../Data/results_with_tags.xlsx')

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

for report_type in df.report_type.unique():
    print("-----",report_type,'-------')
    for structures in df.structure.unique():
        print("-----",structures,'-------')
         # calculate multiclass accuracy, precision, recall
        sub_data = df[(df.structure==structures) & (df.report_type==report_type)]
        y= sub_data["structure_score(GT)"].fillna(1.0).astype(str)
        y_pred = sub_data["predicted_score"].fillna(1.0).astype(str)
        print("accuracy",accuracy_score(y, y_pred).round(2))
        print("precision",precision_score(y, y_pred,average='macro').round(2))
        print("recall",recall_score(y, y_pred,average='macro').round(2))
        print("f1",f1_score(y, y_pred,average='macro').round(2))
        
        
        